Após a execução do notebook rumos_bank_lending_prediction.ipynb e a verificação de que o conjunto de dados foi corretamente carregado, confirmou-se que o pré-processamento está a funcionar adequadamente. O modelo foi treinado sem erros, gerando as respetivas métricas de desempenho e resultando na escolha do modelo Random Forest.

A primeira ação a realizar é, efetivamente, iniciar uma experiência. Cada experiência estará associada a um conjunto de runs. Recomenda-se agrupar na mesma experiência todas as runs que se pretende comparar. Mesmo quando se utilizam modelos distintos, caso estes sejam aplicados aos mesmos dados, será útil compará-los. Por esse motivo, devem ser incluídos na mesma experiência. 
Adicionalmente, sempre que existam novos dados para treino relativamente ao mesmo problema, é recomendável utilizar a mesma experiência, de modo a possibilitar a comparação com os resultados anteriores.

In [19]:
import mlflow

In [ ]:
import os
import mlflow
import mlflow.sklearn
import pandas as pd
import numpy as np
import joblib
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from mlflow.models import infer_signature
from sklearn.model_selection import GridSearchCV

In [21]:
# Obter o diretório raiz do projeto (sobe dois níveis a partir da pasta onde o script está a correr)
root_dir = os.path.abspath(os.path.join(os.getcwd(), "../.."))

# Verifica se estamos dentro de um container Docker
if os.getenv("DOCKER_ENV"):
    dataset_path = "/app/data/lending_data.csv"  # Caminho dentro do Docker
else:
    dataset_path = os.path.join(root_dir, "data/lending_data.csv")  # Caminho local

print("Usando dataset em:", dataset_path)

# Testar se o ficheiro realmente existe antes de carregar
if not os.path.exists(dataset_path):
    raise FileNotFoundError(f" O ficheiro não foi encontrado: {dataset_path}")

# Carregar o dataset
df = pd.read_csv(dataset_path)
seed = 42

Usando dataset em: /Users/dinisguerreiro/Documents/Documentos/Cursos/Data Analysis/Operacionalização de Machine Learning/OML-trabalho-master/rumos_bank/data/lending_data.csv


## Definir a diretoria onde as experiências são guardadas

In [22]:
from pathlib import Path

uri = "http://0.0.0.0:5001"

mlflow.set_tracking_uri(uri)
#mlflow.set_tracking_uri("file:///Users/dinisguerreiro/Documents/Documentos/Cursos/Data Analysis/Operacionalização de Machine Learning/OML-trabalho-master/mlruns")

In [23]:
import mlflow
print("Tracking URI:", mlflow.get_tracking_uri())

Tracking URI: http://0.0.0.0:5001


## Fazer set da experiência "Rumos Bank - Bad Payer Prediction"

In [24]:
mlflow.set_experiment("Rumos Bank - Bad Payer Prediction")

<Experiment: artifact_location='mlflow-artifacts:/245331381971867597', creation_time=1742248655797, experiment_id='245331381971867597', last_update_time=1742248655797, lifecycle_stage='active', name='Rumos Bank - Bad Payer Prediction', tags={}>

## Carregar o dataset

In [25]:
df = pd.read_csv(dataset_path)

## Criar os datasets

In [26]:
# Remover coluna ID (se existir)
if "ID" in df.columns:
    df = df.drop("ID", axis=1)

# Definir features e target
TARGET_COL = "default.payment.next.month"
X = df.drop(columns=[TARGET_COL])
y = df[TARGET_COL]

# Dividir treino e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Normalização
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


## Criar uma run

In [27]:
run = mlflow.start_run(run_name="Random Forest Run")
RUN_ID = run.info.run_uuid
RUN_ID

'730cc90a5e6e421b801968d6b2c99cdc'

## Guardar datasets, modelos, artefactos, métricas e parametros da run

In [28]:
def total_cost(y_true, y_proba, threshold=0.3):
    """
    Calcula o custo total baseado na matriz de confusão.
    Falsos Positivos (FP) custam 1000€ e Falsos Negativos (FN) custam 3000€.
    """
    y_pred = (y_proba >= threshold).astype(int)
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
    return (fp * 1000) + (fn * 3000)

In [31]:
mlflow.end_run()

In [32]:
# Iniciar uma run no MLflow
with mlflow.start_run(run_name="Random Forest Run") as run:
    RUN_ID = run.info.run_uuid

    # Guardar a seed
    mlflow.log_param("seed", 42)

    # Criar e treinar o modelo Random Forest com GridSearch
    rf = RandomForestClassifier(random_state=42, class_weight="balanced")
    param_grid = {'n_estimators': [10, 100, 300, 1000]}
    clf_rf = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy')
    clf_rf.fit(X_train, y_train)

    # Melhor modelo
    best_rf = clf_rf.best_estimator_

    # Fazer previsões
    y_preds_proba = clf_rf.predict_proba(X_test)[:, 1]

    # Definir custo total (se a função existir)
    total_cost_value = total_cost(y_test, y_preds_proba, threshold=0.3)

    # Converter previsões probabilísticas em classes binárias
    y_preds = (y_preds_proba >= 0.3).astype(int)

    # Calcular métricas
    accuracy = accuracy_score(y_test, y_preds)

    # Guardar hiperparâmetros e métricas no MLflow
    mlflow.log_params(clf_rf.best_params_)
    mlflow.log_metric("accuracy", accuracy)
    mlflow.log_metric("total_cost", total_cost_value)

    # Criar assinatura do modelo
    input_example = X_test[:5]
    signature = infer_signature(X_test, best_rf.predict(X_test))

    # Registrar o modelo no MLflow
    mlflow.sklearn.log_model(
        best_rf,
        artifact_path="random_forest_model",
        registered_model_name="RandomForest_Optimized",
        signature=signature,
        input_example=input_example
    )

    print(f"Modelo registrado com sucesso! Accuracy: {accuracy}, Custo Total: {total_cost_value}")


🏃 View run Random Forest Run at: http://0.0.0.0:5001/#/experiments/245331381971867597/runs/a1ce2f6fc5a14fb2923ede44e81b9a2d
🧪 View experiment at: http://0.0.0.0:5001/#/experiments/245331381971867597


NameError: name 'GridSearchCV' is not defined

## Terminar a run

In [30]:
mlflow.end_run()

🏃 View run Random Forest Run at: http://0.0.0.0:5001/#/experiments/245331381971867597/runs/730cc90a5e6e421b801968d6b2c99cdc
🧪 View experiment at: http://0.0.0.0:5001/#/experiments/245331381971867597


## Consultar uma run já concluida

In [ ]:
run = mlflow.get_run(RUN_ID)

In [ ]:
run.data

<RunData: metrics={'accuracy': 0.814, 'total_cost': 2856000.0}, params={'bootstrap': 'True',
 'ccp_alpha': '0.0',
 'class_weight': 'balanced',
 'criterion': 'gini',
 'max_depth': 'None',
 'max_features': 'sqrt',
 'max_leaf_nodes': 'None',
 'max_samples': 'None',
 'min_impurity_decrease': '0.0',
 'min_samples_leaf': '1',
 'min_samples_split': '2',
 'min_weight_fraction_leaf': '0.0',
 'monotonic_cst': 'None',
 'n_estimators': '300',
 'n_jobs': 'None',
 'oob_score': 'False',
 'random_state': '42',
 'seed': '42',
 'verbose': '0',
 'warm_start': 'False'}, tags={'mlflow.log-model.history': '[{"run_id": "8b9c4dd3c68c462a922dfaf006bd5074", '
                             '"artifact_path": "random_forest_model", '
                             '"utc_time_created": "2025-03-13 '
                             '23:38:12.789609", "model_uuid": '
                             '"bd97b35a1c0e49b38520af627fedf210", "flavors": '
                             '{"python_function": {"model_path": "model.pkl", '

In [ ]:
print(mlflow.get_tracking_uri())

file:///Users/dinisguerreiro/Documents/Documentos/Cursos/Data%20Analysis/Operacionalizac%CC%A7a%CC%83o%20de%20Machine%20Learning/OML-trabalho-master/mlruns
